In [2]:
import sys
import re

Задача: Углубить обработку запросов пользователей в чат-боте технической поддержки и категоризировать их по заданным, более узким в отличии от уже работающих, критериям.

Описание текущей ситуации: на данный момент имеются рабочие модели, которые успешно классифицируют запросы пользователей по таким темам как "Обещанный платеж", "Куда делись деньги", "Как подключить интернет", "Хочу поменять тариф", и т.д.

Стоит задача углубить эту классификацию. Далее представлены конкретные задачи от заказчика.

---

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:0px;overflow:hidden;word-break:normal;}
.tg .tg-lboi{border-color:inherit;text-align:left;vertical-align:middle}
.tg .tg-0lax{text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg" style="undefined;table-layout: fixed; width: 759px">
<colgroup>
<col style="width: 27px">
<col style="width: 125px">
<col style="width: 105px">
<col style="width: 171px">
<col style="width: 331px">
</colgroup>
  <tr>
    <th class="tg-lboi">№</th>
    <th class="tg-lboi">Запрос пользователя</th>
    <th class="tg-0lax">Результат текущего фреймворка</th>
    <th class="tg-0lax">Требуемый результат</th>
    <th class="tg-0lax"><center>Комментарий</th>
  </tr>
  <tr>
    <td class="tg-lboi">1</td>
    <td class="tg-lboi">Подключен ли у меня мобильный интернет</td>
    <td class="tg-0lax">Как подключить интернет</td>
    <td class="tg-0lax">Классифицировать как "подключен ли мобильный интернет"</td>
    <td class="tg-0lax">Создадим новый классификатор "Подключен ли \ Есть ли" на том же уровне что и классификатор "Хочу отключить услугу/опцию/подписку". Частицу "ли" будем добавлять к слову перед ней. Модель будет обрабатывать запросы на получение информации.<br>Далее запрос будет классифицироваться на наличие в нем названия услуги</td>
  </tr>
  <tr>
    <td class="tg-lboi">2</td>
    <td class="tg-lboi">Не ноступен обещенный платеж. Почему</td>
    <td class="tg-0lax">Обещанный платеж</td>
    <td class="tg-0lax">Из фразы понятен контекст: "Почему не доступен ОП"</td>
    <td class="tg-0lax">Решение: Не менять модель, а просто после классификации в "Обещанный платеж" проверять содержание сообщения на предмет слов "Почему", "не доступен" и подобных.</td>
  </tr>
  <tr>
    <td class="tg-0pky">3</td>
    <td class="tg-0pky">За что сегодня снялись 15р?</td>
    <td class="tg-0lax">Куда делись деньги</td>
    <td class="tg-0lax">"Начисления за подписка: 21.10.2019 - 15р"</td>
    <td class="tg-0lax">Простое решение, которое приходит в голову - после классификации искать в тексте слова "сегодня", "вчера" и даты, а так же суммы. Можно так же искать и только суммы.</td>
  </tr>
  <tr>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">Я не буду платить за обещанный платеж</td>
    <td class="tg-0lax">Обещанный платеж</td>
    <td class="tg-0lax">Распознать: "отказывается от факта", "хочет отключить"</td>
    <td class="tg-0lax">Случаи с "не хочу", "не буду" классифицировать как "Хочу отключить услугу" - переобучить существующую модель</td>
  </tr>
</table>

Сначала потестим модели в sklearn, затем обучим модель в VW.

Используем для обучения метод мешка слов созданный по методу TF-IDF. Результатом будет являться матрица объекты-признаки с весами, учитывающих как частоту слова в выборке, так и частоту слова в разных текстах (обратная частота).

Лемматизацию проводить не будем, так как очевидно, что преведение слов к их начальным формам помешает отличать разные категории запросов с участием одних и тех же слов.

В качестве тестритуемых моделей будем проверять логистическую регрессию, линейный классификатор, линейный SVM, а так же наивный баессовский классификатор. Все они хорошо показывают себя в задачах классификации текстов. Они хорошо масштабируются, могут работать с большим количеством признаков, на очень больших выборках.

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import metrics

Предположим, что у нас есть выборка с текстами и метками классов и мы разбили ее на обучение и контроль.

Тогда texts - обучающая выборка, labels - вектор соответствующих меток классов

#### Оценка качества работы разных классификаторов

In [ ]:
def text_classifier(vectorizer, transformer, classifier):
    return Pipeline(
            [("vectorizer", vectorizer),
            ("transformer", transformer),
            ("classifier", classifier)]
        )

In [ ]:
# Тестируем классификаторы, оцениваем на кросс-валидации с помощью f-меры

f1 = metrics.make_scorer(metrics.f1_score)

for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    print(cross_val_score(text_classifier(CountVectorizer(), TfidfTransformer(), 
                                          clf()), texts, labels, scoring=f1).mean())
    print("\n")

#### Оценка использования n-грамм

Далее применим n-граммы и проверим вариацию качества на кросс-валидации. Потестим на только биграммах, затем униграммах и биграммах, затем униграммах, биграммах и триграммах.

Кроме того, учитывая наличие большого количества опечаток в текстах, в дальнейшем полезно попробовать применить буквенные n-граммы, которые могут помочь улучшить классификацию текстов с опечатками.

In [ ]:
for clf in [LogisticRegression, LinearSVC, SGDClassifier]:
    print(clf)
    for i in [[2,2], [1,2], [1,3]]:
        print('Ngram range:', i)
        res = cross_val_score(text_classifier(CountVectorizer(ngram_range=(i)), TfidfTransformer(), 
                                          clf()), texts, labels, scoring=f1, cv=3).mean()
        print('Cross-val-score f1: %.4f\n' % res)
    

Далее провалидируем лучшую модель на отложенной выборке. Дальнейшие шаги зависят от того, обнаруживается ли переобучение на этапе валидации и устраивает ли качество работы алгоритма.

#### Vowpal Wabbit

В случае дальнейшего использования линейных моделей будем пользоваться библиотекой Vowpal Wabbit которая широко используется для работы с текстом и онлайн обучения моделей. В ней так же используются линейные модели.

Приведем наши данные к формату Vowpal Wabbit, при этом оставляя только слова не короче 2 символов. На данном этапе уместно проводить предобработку данных: применить регулярные выражения, удалить стоп-слова.

In [ ]:
def to_vw_format(document, label=None):
    return str(label or '') + ' |text ' + ' '.join(re.findall('\w{2,}', document.lower())) + '\n'

to_vw_format(text, 1 if target == 1 else -1)

In [ ]:
# Приведем выборку к формату VW

train_documents, test_documents, train_labels, test_labels = \
    train_test_split(all_documents, all_targets, random_state=7)
    
with open('train.vw', 'w') as train_data:
    for text, target in zip(train_documents, train_labels):
        try:
            vw_train_data.write(to_vw_format(text, target))
        except:
            err = to_vw_format(text, target)
            break
with open('test.vw', 'w', encoding="utf-8") as test_data:
    for text in test_documents:
        vw_test_data.write(to_vw_format(text))

Запустим Vowpal Wabbit на сформированном файле. Зададим функцию потерь, которая показала лучшее значение на предыдущих тестах, пока пусть это будет  значение hinge (линейный SVM), добавим n-граммы. Построенную модель сохраним в соответствующий файл model.vw

In [ ]:
!vw -d train_data.vw --loss_function hinge --ngram 2 -f news_data/model.vw

Применим обученную модель на тестовой выборке, сохраним результат в файл

In [ ]:
!vw -i model.vw -t -d test.vw -p test_predictions.txt

Загрузим полученные предсказания, вычислим AUC

In [ ]:
test_prediction = None
with open('../../data/news_data/20news_test_predictions.txt') as pred_file:
    test_prediction = [float(label) 
                             for label in pred_file.readlines()]

auc = roc_auc_score(test_labels, test_prediction)

VW так же удобно использовать при многоклассовой классификации, для этого при обучении на соответсвующей выборке (классы должны быть нумерованы начиная с 1) передается параметр --oaa ('one against all)

Настраиваемые гиперпараметры, которые оказывают существенное влияние на качество модели:


- темп обучения (-l, по умолчанию 0.5) – коэффициент перед изменением весов модели при каждом изменении
- степень убывания темпа обучения (--power_t, по умолчанию 0.5) – на практике проверено, что если темп обучения уменьшается при увеличении числа итераций стохастического градиентного спуска, то минимум функции находится лучше
- функция потерь (--loss_function) – от нее, по сути, зависит обучаемый алгоритм. Про функции mпотерь в документации
- регуляризация (-l1) – тут надо обратить внимание на то, что в VW регуляризация считается для каждого объекта, поэтому коэффициенты регуляризации обычно берутся малыми, около  10−20.